In [2]:

pip install findspark


Note: you may need to restart the kernel to use updated packages.


In [36]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

spark

In [45]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum

from functools import reduce
from operator import add



In [46]:

import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MyApp").master("local[*]").getOrCreate()


In [47]:

# Load CSV files with header and infer schema
urban_df = spark.read.option("header", "true").option("inferSchema", "true").csv("C:/Users/kchangde/Downloads/FINAL_POPULATION_URBAN 2.csv")

rural_df = spark.read.option("header", "true").option("inferSchema", "true").csv("C:/Users/kchangde/Downloads/FINAL_POPULATION_RURAL 2.csv")


In [48]:

# Rename column in urban_df to match rural_df
urban_renamed = urban_df.withColumnRenamed("DEVELOPMENT_CATEGORY", "DEVELOPMENT")


In [49]:
# Create list of year columns
year_cols = [col(f"D_{year}") for year in range(1950, 2051, 5)]


In [54]:
# Use reduce to sum the columns
urban_with_total = urban_renamed.withColumn("TOTAL_POPULATION", reduce(add, year_cols))
rural_with_total = rural_df.withColumn("TOTAL_POPULATION", reduce(add, year_cols))


In [55]:

# Merge datasets
merged_df = urban_with_total.unionByName(rural_with_total)



In [57]:

# Group by country and sum total population
country_total = merged_df.groupBy("COUNTRY").agg(_sum("TOTAL_POPULATION").alias("TOTAL_POPULATION"))


In [58]:

print("Country-wise Total Population:")
country_total.show()


Country-wise Total Population:
+--------------------+----------------+
|             COUNTRY|TOTAL_POPULATION|
+--------------------+----------------+
|                Chad|        228765.0|
|            Anguilla|           251.0|
|            Paraguay|        116634.0|
|               Yemen|        493521.0|
|             Senegal|        250071.0|
|              Sweden|        190057.0|
|             Tokelau|            32.0|
|            Kiribati|          1831.0|
|   Republic of Korea|        861101.0|
|              Guyana|         15025.0|
|             Eritrea|        105035.0|
|         Philippines|       1696445.0|
|            Djibouti|         15509.0|
|Occupied Palestin...|         82537.0|
|               Tonga|          2078.0|
|            Malaysia|        502298.0|
|           Singapore|         82304.0|
|                Fiji|         15635.0|
|              Turkey|       1282240.0|
|United States Vir...|          1829.0|
+--------------------+----------------+
only show

In [60]:
country_total.write.mode("overwrite").orc("C:/Users/kchangde/Desktop/PY2.orc")
